In [1]:
import random
import numpy as np
import pandas as pd
import time

In [2]:
# Se crea la matriz de pesos y recompensas de los cargamentos
weight = [61, 58, 92, 50, 108, 83, 93, 101, 54, 50, 72, 51, 100, 108, 91, 112, 66, 58, 110, 73]
reward = [1100, 1147, 1442, 1591, 1078, 1385, 1777, 1196, 1753, 1371, 1517, 1675, 1193, 1177, 1365, 1143, 1314, 1526, 1470, 1605]

In [3]:
#Esta función calcula el peso de cada agente en la población
def calc_weight(chromosome, weight):
        return sum([chromosome[i] * weight[i] for i in range(len(chromosome))])
    
def calc_reward(chromosome, reward):
    return sum([chromosome[i] * reward[i] for i in range(len(chromosome))])

def calc_fitness(weight,reward):
    max_weight = 800
    if weight > max_weight:
        return 0
    else:
        return reward

In [4]:
# Se crea la clase de agente, cuenta con sus cromosomas, peso y recompensa
class Agent():
    def __init__(self,weight_list, reward_list, num_genes=20):
        self.chromosome = [bool(random.choice([0, 1])) for _ in range(num_genes)]
        self.num_genes = num_genes
        self.weight = calc_weight(self.chromosome, weight_list)
        self.reward = calc_reward(self.chromosome, reward_list)
        self.fitness = calc_fitness(self.weight, self.reward)
        if self.weight >= 800:
            self.generate_valid_chromosome(weight_list, reward_list)
                
        
    #Se crea la función de mutar el cual elige un gen al azar y lo cambia
    def mutate(self, weight_list, reward_list):
        max_weight = 800
        while True:
            is_mutated = [random.random() for _ in range(self.num_genes)]
            self.chromosome = [not self.chromosome[i] if is_mutated[i] < 0.05 else self.chromosome[i] for i in range(self.num_genes)]
            self.weight = calc_weight(self.chromosome, weight_list)
            if self.weight <= max_weight:
                self.reward = calc_reward(self.chromosome, reward_list)
                self.fitness = calc_fitness(self.weight, self.reward)
                break
    
    def generate_valid_chromosome(self, weight_list, reward_list):
        max_weight = 800
        while True:
            self.chromosome = [bool(random.choice([0, 1])) for _ in range(self.num_genes)]
            self.weight = calc_weight(self.chromosome, weight_list)
            self.reward = calc_reward(self.chromosome, reward_list)
            self.fitness = calc_fitness(self.weight, self.reward)
            if self.weight <= max_weight:
                break

In [5]:
#Se toma la mitad de la población con mejor fitness
def selection(population):
    sorted_population = sorted(population, key=lambda x: x.fitness, reverse=True)
    return sorted_population[:len(population)//2]
    
#Se realiza el cruce de los cromosomas de los agentes
def crossover(population):
    offspring = []
    
    # Convert the elements list to a numpy array
    arpopulation = np.array(population)
    
    # Shuffle the elements using numpy's shuffle
    np.random.shuffle(arpopulation)

    # Check if the number of elements is even
    if len(arpopulation) % 2 != 0:
        raise ValueError("The number of elements must be even to form pairs.")
    
    # Reshape the array into pairs
    arparents = arpopulation.reshape(-1, 2)
    
    # Convert the numpy array of pairs back to a list of lists
    parents = arparents.tolist()
    
    for parent in parents:
        parent1 = parent[0] 
        parent2 = parent[1]
        child1 = Agent(weight, reward)
        child2 = Agent(weight, reward)
        child3 = Agent(weight, reward)
        child4 = Agent(weight, reward)
        child1.chromosome = parent1.chromosome[:10] + parent2.chromosome[10:]
        child2.chromosome = parent2.chromosome[:10] + parent1.chromosome[10:] 
        child3.chromosome = parent2.chromosome[10:] + parent2.chromosome[:10]
        child4.chromosome = parent1.chromosome[10:] + parent1.chromosome[:10]
        childs = [child1, child2, child3, child4]
        
        # Se toman solo los dos mejores hijos de cada cruce
        mutated_childs = mutation(childs)
        #Eliminar los hijos que no cumplan con el peso
        feasible_childs = filter(lambda x: x.weight <= 800, mutated_childs)
        feasible_childs = sorted(feasible_childs, key=lambda x: x.fitness)
        offspring.extend(feasible_childs)
        
    return offspring
# Se realiza la probabilidad de mutación
def mutation(population):
    for entity in population:
        entity.mutate(weight, reward)
    return population

In [6]:
performance = {}

# Se inicializa la población
population_size = 300
population = [Agent(weight,reward, num_genes=20) for _ in range(population_size)]
counter = 0
past_agent = 0
epoch = 0
#Número de epocas
all_time_best_agent = max(population, key=lambda x: x.fitness)

while counter < 10:
    start_time = time.time()
    epoch += 1
    purged_population = selection(population)
    babies = crossover(purged_population)
    while len(babies) < len(population)//2:
        babies.append(crossover(purged_population))
    babies = babies[:len(population)//2]
    purged_population.extend(babies)
    population = purged_population
    
    current_best_agent = max(population, key=lambda x: x.fitness)
    if current_best_agent.fitness <= all_time_best_agent.fitness:
        counter += 1
    else:
        counter = 0
        all_time_best_agent = current_best_agent

    end_time = time.time()

    stats = {
        'Epoch': epoch,
        'Best Agent Fitness': current_best_agent.fitness,
        'Best Agent Reward': current_best_agent.reward,
        'Best Agent Weight': current_best_agent.weight,
        'Execution Time': end_time - start_time
    }

    performance[epoch] = stats

    # print("Epoca:"+ str(epoch))
    # # print("Counter:"+ str(counter))
    # print("Este es el fitness mejor agente: " + str(current_best_agent.fitness))
    # print("Este es el reward mejor agente: " + str(current_best_agent.reward))
    # print("Este es el peso mejor agente: " + str(current_best_agent.weight))

print("RESULTADOS FINALES")
print("Este es el fitness mejor agente: " + str(all_time_best_agent.fitness))
print("Este es el reward mejor agente: " + str(all_time_best_agent.reward))
print("Este es el peso mejor agente: " + str(all_time_best_agent.weight))
print("Este es el cromosoma mejor agente: " + str([index + 1 for index, value in enumerate(all_time_best_agent.chromosome) if value == 1]))


RESULTADOS FINALES
Este es el fitness mejor agente: 18103
Este es el reward mejor agente: 18103
Este es el peso mejor agente: 800
Este es el cromosoma mejor agente: [2, 3, 4, 6, 7, 9, 10, 11, 12, 17, 18, 20]


In [7]:
pd.DataFrame(performance).T

,Epoch,Best Agent Fitness,Best Agent Reward,Best Agent Weight,Execution Time
1,1.0,15941.0,15941.0,798.0,0.037616
2,2.0,16180.0,16180.0,794.0,0.047184
3,3.0,16561.0,16561.0,796.0,0.046272
4,4.0,16561.0,16561.0,796.0,0.031662
5,5.0,16561.0,16561.0,796.0,0.031815
6,6.0,16561.0,16561.0,796.0,0.028596
7,7.0,16561.0,16561.0,796.0,0.034764
8,8.0,16625.0,16625.0,769.0,0.028373
9,9.0,16804.0,16804.0,761.0,0.023346
10,10.0,16804.0,16804.0,761.0,0.029184
